In [30]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
cancer_df = pd.DataFrame(data = cancer.data,columns=cancer.feature_names)
cancer_df['target'] = cancer.target
cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


# Feature Selection techniques
## sklearn - SelectFromModel
## sklearn - RFE(i.e, Recursive Feature Elimination)

# Model Building

In [3]:
X = cancer_df.iloc[:,:-1]
y = cancer_df.iloc[:,30:]


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=12,test_size=.20,stratify=y)

In [5]:
X_train.shape,y_train.shape

((455, 30), (455, 1))

### Feature Selection
#### 1. SelectFromModel

In [6]:
from sklearn.feature_selection import SelectFromModel,RFE
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [7]:
rf =RandomForestClassifier(max_depth=10,random_state=12)

In [8]:
select_from_model=SelectFromModel(estimator=rf,max_features =None)
select_from_model.fit(X_train,y_train)

C:\Users\KEERTHI KUMAR\anaconda3\lib\site-packages\sklearn\feature_selection\_from_model.py:234: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


SelectFromModel(estimator=RandomForestClassifier(max_depth=10, random_state=12))

In [9]:
select_from_model.get_support() 

array([False, False, False,  True, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False,  True,  True, False, False, False,
        True, False, False])

In [10]:
X_train.columns[select_from_model.get_support()]

Index(['mean area', 'mean concave points', 'area error', 'worst radius',
       'worst perimeter', 'worst area', 'worst concave points'],
      dtype='object')

In [11]:
len(X_train.columns[select_from_model.get_support()])

7

In [12]:
rf.fit(X_train,y_train)

<ipython-input-12-d6d9475b85dc>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)


RandomForestClassifier(max_depth=10, random_state=12)

In [13]:
y_pred_test = rf.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(y_test,y_pred_test)

0.956140350877193

# by selectFromModel

In [16]:
imp_ft_train = select_from_model.transform(X_train)
imp_ft_test=select_from_model.transform(X_test)

In [17]:
rf.fit(imp_ft_train,y_train)

<ipython-input-17-9bfcb16ede8b>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(imp_ft_train,y_train)


RandomForestClassifier(max_depth=10, random_state=12)

In [18]:
y_pred_test2=rf.predict(imp_ft_test)

In [19]:
accuracy_score(y_test,y_pred_test2)

0.9473684210526315

## Looping

In [22]:
from sklearn.metrics import precision_score,recall_score,confusion_matrix

In [28]:
def rf_classifier(X_train,X_test,y_train,y_test):
    rf = RandomForestClassifier()
    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    print('accuracy score:', round(accuracy_score(y_test,y_pred),4))
    print('precision score:', round(precision_score(y_test,y_pred),4))
    print('recall  score:', round(recall_score(y_test,y_pred),4))
    print('confussion matrix:\n',confusion_matrix(y_test,y_pred))

In [34]:
%%time
rf_classifier(X_train,X_test,y_train,y_test)

accuracy score: 0.9737
precision score: 0.9726
recall  score: 0.9861
confussion matrix:
 [[40  2]
 [ 1 71]]
Wall time: 202 ms


In [32]:
rf_classifier(imp_ft_train,imp_ft_test,y_train,y_test)

accuracy score: 0.9474
precision score: 0.9583
recall  score: 0.9583
confussion matrix:
 [[39  3]
 [ 3 69]]


### By rfe

In [165]:
rfe = RFE(estimator=rf,n_features_to_select=12)
rfe.fit(X_train,y_train)

SyntaxError: positional argument follows keyword argument (<ipython-input-165-1f92fb61dbba>, line 1)

In [44]:
rfe.get_support()

array([ True, False,  True,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True,  True, False, False,  True,
        True, False, False])

In [45]:
X_train.columns[rfe.get_support()]

Index(['mean radius', 'mean perimeter', 'mean area', 'mean concavity',
       'mean concave points', 'area error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst concavity',
       'worst concave points'],
      dtype='object')

In [46]:
len(X_train.columns[rfe.get_support()])

12

In [163]:
X_train_trans_rfe =rfe.transform(X_train)
X_test_trans_rfe = rfe.transform(X_test)

ValueError: X has a different shape than during fitting.

In [50]:
rf_classifier(X_train_trans_rfe,X_test_trans_rfe,y_train,y_test)

accuracy score: 0.9561
precision score: 0.9718
recall  score: 0.9583
confussion matrix:
 [[40  2]
 [ 3 69]]


### Pick up the imp features from gradient boosting to build rf classifier

In [52]:
rfe = RFE(estimator= GradientBoostingClassifier(random_state=12),n_features_to_select=12)
rfe.fit(X_train,y_train)

RFE(estimator=GradientBoostingClassifier(random_state=12),
    n_features_to_select=12)

In [53]:
rfe.get_support()

array([False,  True, False,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False,  True, False,
       False,  True, False,  True, False,  True, False, False,  True,
        True,  True, False])

In [55]:
X_train.columns[rfe.get_support()]

Index(['mean texture', 'mean area', 'mean concavity', 'mean concave points',
       'area error', 'concavity error', 'fractal dimension error',
       'worst texture', 'worst area', 'worst concavity',
       'worst concave points', 'worst symmetry'],
      dtype='object')

In [56]:
len(X_train.columns[rfe.get_support()])

12

In [162]:
rf_classifier(X_train_trans_rfe,X_test_trans_rfe,y_train,y_test)

accuracy score: 0.9386
precision score: 0.971
recall  score: 0.9306
confussion matrix:
 [[40  2]
 [ 5 67]]


## To Design optimal no. of features to get better accuracy

In [61]:
for i in range(1,31):
    rfe = RFE(estimator= GradientBoostingClassifier(random_state=12),n_features_to_select=i)
    rfe.fit(X_train,y_train)
    X_train_trans_rfe =rfe.transform(X_train)
    X_test_trans_rfe = rfe.transform(X_test)
    print('selected features:',i)
    rf_classifier(X_train_trans_rfe,X_test_trans_rfe,y_train,y_test)
    print('=======================================================') 

selected features: 1
accuracy score: 0.8158
precision score: 0.8696
recall  score: 0.8333
confussion matrix:
 [[33  9]
 [12 60]]
selected features: 2
accuracy score: 0.9123
precision score: 0.9306
recall  score: 0.9306
confussion matrix:
 [[37  5]
 [ 5 67]]
selected features: 3
accuracy score: 0.9211
precision score: 0.9315
recall  score: 0.9444
confussion matrix:
 [[37  5]
 [ 4 68]]
selected features: 4
accuracy score: 0.9561
precision score: 0.9589
recall  score: 0.9722
confussion matrix:
 [[39  3]
 [ 2 70]]
selected features: 5
accuracy score: 0.9474
precision score: 0.9714
recall  score: 0.9444
confussion matrix:
 [[40  2]
 [ 4 68]]
selected features: 6
accuracy score: 0.9386
precision score: 0.9577
recall  score: 0.9444
confussion matrix:
 [[39  3]
 [ 4 68]]
selected features: 7
accuracy score: 0.9561
precision score: 0.9718
recall  score: 0.9583
confussion matrix:
 [[40  2]
 [ 3 69]]
selected features: 8
accuracy score: 0.9737
precision score: 0.9726
recall  score: 0.9861
confuss

In [77]:
rfe = RFE(estimator= GradientBoostingClassifier(random_state=te=12,),n_features_to_select=8)
rfe.fit(X_train,y_train)

RFE(estimator=GradientBoostingClassifier(random_state=12),
    n_features_to_select=8)

In [78]:
X_train.columns[rfe.get_support()]

Index(['mean texture', 'mean concavity', 'mean concave points', 'area error',
       'worst texture', 'worst area', 'worst concavity',
       'worst concave points'],
      dtype='object')

In [142]:
X_train_trans_rfe =rfe.transform(X_train)
X_test_trans_rfe = rfe.transform(X_test)
rf_classifier(X_train_trans_rfe,X_test_trans_rfe,y_train,y_test)
print('=======================================================') 

accuracy score: 0.9474
precision score: 0.9583
recall  score: 0.9583
confussion matrix:
 [[39  3]
 [ 3 69]]
